In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='./data', train=True, download=True, transform=transform),
    batch_size=64, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='./data', train=False, transform=transform),
    batch_size=1000, shuffle=False
)


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        return self.fc(x)

model = Net().to(device)


In [4]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
losses = []

for epoch in range(5):
    model.train()
    epoch_loss = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    losses.append({'epoch': epoch+1, 'loss': avg_loss})
    print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}")


Epoch 1: Loss = 0.2693
Epoch 2: Loss = 0.1137
Epoch 3: Loss = 0.0816
Epoch 4: Loss = 0.0606
Epoch 5: Loss = 0.0504


In [5]:
df = pd.DataFrame(losses)
df.to_csv("training_losses.csv", index=False)
df


,epoch,loss
0,1,0.269267
1,2,0.113733
2,3,0.081604
3,4,0.060637
4,5,0.050405


In [6]:
import torch.onnx

dummy_input = torch.randn(1, 1, 28, 28).to(device)
torch.onnx.export(model, dummy_input, "mnist_model.onnx",
                  input_names=["input"], output_names=["output"],
                  dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}})

print("Modèle exporté en mnist_model.onnx ✅")


Modèle exporté en mnist_model.onnx ✅


In [7]:
import onnx

model = onnx.load("mnist_model.onnx")
print([input.name for input in model.graph.input])


['input']


In [ ]:
import onnxruntime as ort
import numpy as np
from torchvision import datasets, transforms

# Charger le modèle ONNX
session = ort.InferenceSession("mnist_model.onnx")

# Préparation de la transformation identique à celle du training
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Ajuste selon ton training !
])

# Charger jeu de test MNIST (download si besoin)
test_dataset = datasets.MNIST(root='.', train=False, download=True, transform=transform)

# Prendre une image test avec son label (exemple: index 1)
img, label = test_dataset[1]

# Préparer l'entrée pour le modèle ONNX
input_name = session.get_inputs()[0].name

# MNIST : batch_size=1, canaux=1, hauteur=28, largeur=28
input_data = img.numpy().reshape(1, 1, 28, 28).astype(np.float32)

# Inférence
outputs = session.run(None, {input_name: input_data})

# Résultat brut (logits)
logits = outputs[0]

# Affichage des logits et prédiction finale
print("Logits:", logits)
predicted_label = np.argmax(logits)
print(f"Prédiction du modèle : {predicted_label}")
print(f"Label réel : {label}")
